In [20]:
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import textwrap
from tqdm import tqdm
import matplotlib.pyplot as plt
import re
from typing import TypedDict, List, Optional
tqdm.pandas()
from datetime import datetime, timedelta
import ast
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('spanish'))
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("spanish")
import uuid
from google.cloud import storage
# --------------- LLAMA --------------------------
from llama_index.readers.web import HyperbrowserWebReader
from llama_index.core.schema import Document
from llama_index.core.extractors import TitleExtractor
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.core.ingestion import IngestionPipeline
from llama_index.vector_stores.elasticsearch import ElasticsearchStore as le
from llama_index.core import StorageContext, VectorStoreIndex

In [19]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Omen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Omen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Omen\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

### Variables de entorno

In [2]:
with open("APPI OPENAI.txt") as archivo:
  apikey_openai = archivo.read()
os.environ["OPENAI_API_KEY"] = apikey_openai

In [3]:
with open("APPI Hyperbrowser.txt") as archivo:
  apikey_hyper = archivo.read()
os.environ["HYPERBROWSER_API_KEY"] = apikey_hyper

In [3]:
with open("postgrest.txt") as archivo:
  uribd = archivo.read()

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credenciales.json"

In [5]:
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "gcpaiagent"

### Importamos base de datos

In [7]:
noticias_raw1 = pd.read_excel('Noticias_Peru21.xlsx')
noticias_raw2 = pd.read_excel('Noticias_correo.xlsx')
noticias_raw3 = pd.read_excel('Noticias_ElComercio.xlsx')
noticias_raw4 = pd.read_excel('Noticias_Exitosa.xlsx')

In [8]:
noticias_raw = pd.concat([noticias_raw1,noticias_raw2,noticias_raw3,noticias_raw4])

In [9]:
noticias_raw.isnull().sum()

url             0
titulo          0
texto           7
fecha           0
distrito        0
departamento    0
pais            0
dtype: int64

In [10]:
noticias_raw['fecha'] = pd.to_datetime(noticias_raw['fecha'], format='%d/%m/%Y', errors='coerce')
noticias_raw = noticias_raw.dropna()
noticias_raw['fecha'] = pd.to_datetime(noticias_raw['fecha'], format='%d/%m/%Y').dt.strftime('%Y/%m/%d')

In [11]:
noticias_raw.isnull().sum()

url             0
titulo          0
texto           0
fecha           0
distrito        0
departamento    0
pais            0
dtype: int64

In [12]:
noticias_raw = noticias_raw.reset_index(drop=True)
noticias_raw

,url,titulo,texto,fecha,distrito,departamento,pais
0,https://peru21.pe/peru/medios-deberan-tener-fr...,Medios deberán tener 'franja educativa' para a...,"El Congreso aprobó la Ley N° 32351, que modifi...",2025/05/27,['Independencia'],['La Libertad'],"[""perú""]"
1,https://peru21.pe/peru/chiclayo-detienen-profe...,Chiclayo: detienen a profesor por acosar sexua...,La madre de la menor reportó mensajes y fotogr...,2025/04/16,[],[],"[""perú""]"
2,https://peru21.pe/lima/miraflores-sujeto-que-h...,Miraflores: Sujeto violento que hirió con fuen...,"Jueza Susana Quispe, del Cuarto Juzgado de Inv...",2025/03/26,"['Lima', 'Miraflores']",['Lima'],"[""perú""]"
3,https://peru21.pe/lima/atacante-de-miraflores-...,Atacante de Miraflores también agredió a su ex...,Julio Diego Aurelio Solari Zapata fue denuncia...,2025/03/24,"['Miraflores', 'San Isidro']",[],"[""perú""]"
4,https://peru21.pe/mundo/la-senal-de-auxilio-gl...,La señal de auxilio global que puede salvar la...,El sencillo gesto de abrir la palma de la mano...,2025/02/20,[],[],"[""canadá"", ""méxico"", ""estados unidos"", ""perú""]"
...,...,...,...,...,...,...,...
1009,https://www.exitosanoticias.pe/actualidad/denu...,"Denuncia a Christian Cueva: ""El intento de fem...",En el marco de la denuncia contra Christian Cu...,2024/08/20,[],[],"[""perú""]"
1010,https://www.exitosanoticias.pe/espectaculos/ch...,Christian Cueva iría a prisión por feminicidio...,El futbolista Christian Cueva se ve en serios ...,2024/08/19,[],[],[]
1011,https://www.exitosanoticias.pe/espectaculos/pa...,Pamela López denuncia que Christian Cueva la g...,"Pamela López denunció que su esposo, Christian...",2024/08/19,[],[],[]
1012,https://www.exitosanoticias.pe/actualidad/salg...,"""Salga de su zona de confort"": Sasieta critica...",Ante el reciente caso de feminicidio en Chorri...,2024/08/14,['Chorrillos'],[],"[""perú""]"


In [13]:
noticias_raw['pais'] = noticias_raw['pais'].progress_apply(
    lambda x: ['perú'] if len(ast.literal_eval(x)) == 0 else ast.literal_eval(x)
)

100%|██████████| 1014/1014 [00:00<00:00, 110193.39it/s]


In [14]:
noticias_raw['distrito'] = noticias_raw['distrito'].progress_apply(
    lambda x: [] if len(ast.literal_eval(x)) == 0 else ast.literal_eval(x)
)

100%|██████████| 1014/1014 [00:00<00:00, 113951.83it/s]


In [15]:
noticias_raw['departamento'] = noticias_raw['departamento'].progress_apply(
    lambda x: [] if len(ast.literal_eval(x)) == 0 else ast.literal_eval(x)
)

100%|██████████| 1014/1014 [00:00<00:00, 115867.28it/s]


In [16]:
noticias_raw['departamento'] = noticias_raw.progress_apply(
    lambda x: ['Lima'] if len(x['distrito']) != 0 and len(x['departamento']) == 0 else x['departamento'],
    axis=1
)

100%|██████████| 1014/1014 [00:00<00:00, 177091.28it/s]


In [17]:
noticias_raw['test'] = noticias_raw['pais'].progress_apply(
    lambda x: True if 'perú' in x else False
)

100%|██████████| 1014/1014 [00:00<?, ?it/s]


In [18]:
noticias_raw = noticias_raw[noticias_raw['test']]
noticias_raw = noticias_raw.reset_index(drop=True)

In [21]:
noticias_raw

,url,titulo,texto,fecha,distrito,departamento,pais,test
0,https://peru21.pe/peru/medios-deberan-tener-fr...,Medios deberán tener 'franja educativa' para a...,"El Congreso aprobó la Ley N° 32351, que modifi...",2025/05/27,[Independencia],[La Libertad],[perú],True
1,https://peru21.pe/peru/chiclayo-detienen-profe...,Chiclayo: detienen a profesor por acosar sexua...,La madre de la menor reportó mensajes y fotogr...,2025/04/16,[],[],[perú],True
2,https://peru21.pe/lima/miraflores-sujeto-que-h...,Miraflores: Sujeto violento que hirió con fuen...,"Jueza Susana Quispe, del Cuarto Juzgado de Inv...",2025/03/26,"[Lima, Miraflores]",[Lima],[perú],True
3,https://peru21.pe/lima/atacante-de-miraflores-...,Atacante de Miraflores también agredió a su ex...,Julio Diego Aurelio Solari Zapata fue denuncia...,2025/03/24,"[Miraflores, San Isidro]",[Lima],[perú],True
4,https://peru21.pe/mundo/la-senal-de-auxilio-gl...,La señal de auxilio global que puede salvar la...,El sencillo gesto de abrir la palma de la mano...,2025/02/20,[],[],"[canadá, méxico, estados unidos, perú]",True


### Ingesta

In [22]:
reader = HyperbrowserWebReader(api_key=apikey_hyper)
def safe_load(url):
    try:
        return reader.load_data([url])
    except Exception as e:
        print(f"Error al cargar {url}: {e}")
        return None  # o [] si prefieres una lista vacía

noticias_raw['document_loader'] = noticias_raw['url'].progress_apply(safe_load)

100%|██████████| 5/5 [03:32<00:00, 42.57s/it]


In [23]:
noticias_raw.document_loader[0][0].metadata

{'url': 'https://peru21.pe/peru/medios-deberan-tener-franja-educativa-para-apoyar-lucha-contra-la-violencia-hacia-las-mujeres/',
 'title': "Medios deberán tener 'franja educativa' para apoyar lucha contra la violencia hacia las mujeres",
 'author': 'Perú 21',
 'og:url': 'https://peru21.pe/peru/medios-deberan-tener-franja-educativa-para-apoyar-lucha-contra-la-violencia-hacia-las-mujeres/',
 'robots': 'follow',
 'og:type': 'article',
 'abstract': 'Noticias del Perú y del mundo en peru21.pe - Ultimas noticias de política, deportes, espectáculos, fútbol, economía, mundo, tecnología, cultura, ciencia y más...',
 'keywords': 'Violencia contra la mujer,Violencia familiar,Medios de comunicación,Libertad de prensa',
 'og:image': 'http://peru21.pe/sites/default/efsfiles/styles/amp_1200x675_16_9/public/2025-05/television.jpg?itok=7pZmAQmf',
 'og:title': "Medios deberán tener 'franja educativa' para apoyar lucha contra la violencia hacia las mujeres",
 'viewport': 'width=device-width, initial-scal

In [24]:
print(textwrap.fill(noticias_raw.document_loader[0][0].text,width=90))

Medios deberán tener 'franja educativa' para apoyar lucha contra la violencia hacia las
mujeres[Pasar al contenido principal](https://peru21.pe/peru/medios-deberan-tener-franja-
educativa-para-apoyar-lucha-contra-la-violencia-hacia-las-mujeres#main-content)
[![](https://peru21.pe/white-logo.png)](https://peru21.pe/)  [Ashley
Vargas](https://peru21.pe/noticias/ashley-vargas-mendoza/)  [Crisis en
Pataz](https://peru21.pe/noticias/pataz/)  [Copa
Libertadores](https://peru21.pe/noticias/copa-libertadores/)
[Huariqueando](https://peru21.pe/noticias/huariqueando/)
[#DileNoalGotaaGota](https://peru21.pe/noticias/dilenoalgotaagota/)
[#GenéricosEficacesYa](https://peru21.pe/noticias/genericos-eficaces-ya/)
[#NiñoPeruanoComeSano](https://peru21.pe/noticias/ninoperuanocomesano/)  PUBLICIDAD
[Perú](https://peru21.pe/peru/)  - [Compartir en Whatsapp](whatsapp://send?text=Medios%20d
eber%C3%A1n%20tener%20%27franja%20educativa%27%20para%20apoyar%20lucha%20contra%20la%20vio
lencia%20hacia%20las%20muje

In [25]:
print(textwrap.fill(noticias_raw.texto[0],width=90))

El Congreso aprobó la Ley N° 32351, que modifica la Ley 30364, y obliga a los medios de
comunicación a incluir contenidos educativos contra la violencia familiar y garantizar un
tratamiento respetuoso de las víctimas. Los medios de comunicación del país están
obligadosa participar de la lucha contra laviolencia hacia las mujeresy los integrantes
del grupo familiar. Así lo establece la Ley N° 32351, promulgada por el Congreso y
difundida este martes en El Peruano. La norma modifica el artículo 46 de la Ley 30364, e
introduce disposiciones para que los medios garanticen un enfoque de respeto a los
derechos humanos, la libertad y la dignidad de las víctimas de violencia. También exige
especial cuidado en el tratamiento gráfico de los contenidos que aborden estos casos. Uno
de los principales cambios es la obligación de ceder, de forma gratuita, el 10 % de su
programación durante el horario de protección familiar a instituciones públicas del
Sistema Nacional para la Prevención, Sanción y E

In [26]:
def set_metadata_replace(row):
    if row['document_loader'] is not None:
        try:
            old_doc = row['document_loader'][0]
            new_doc = Document(
                text=row['texto'],
                metadata={
                    **old_doc.metadata,
                    'fecha_publicacion': row['fecha'],
                    'distrito': row['distrito'],
                    'departamento': row['departamento'],
                    'title' : row['titulo']
}
            )
            row['document_loader'] = [new_doc]
        except Exception as e:
            print(f"Error al reemplazar Document: {e}")
    return row

In [27]:
noticias_raw = noticias_raw.progress_apply(set_metadata_replace, axis=1)

100%|██████████| 5/5 [00:00<00:00, 1647.28it/s]


In [28]:
noticias_raw

,url,titulo,texto,fecha,distrito,departamento,pais,test,document_loader
0,https://peru21.pe/peru/medios-deberan-tener-fr...,Medios deberán tener 'franja educativa' para a...,"El Congreso aprobó la Ley N° 32351, que modifi...",2025/05/27,[Independencia],[La Libertad],[perú],True,[Doc ID: b44ade9d-994d-44df-8b32-1bc358fe410a\...
1,https://peru21.pe/peru/chiclayo-detienen-profe...,Chiclayo: detienen a profesor por acosar sexua...,La madre de la menor reportó mensajes y fotogr...,2025/04/16,[],[],[perú],True,[Doc ID: 9b63d202-209b-43b6-91e9-599ad8cebdc6\...
2,https://peru21.pe/lima/miraflores-sujeto-que-h...,Miraflores: Sujeto violento que hirió con fuen...,"Jueza Susana Quispe, del Cuarto Juzgado de Inv...",2025/03/26,"[Lima, Miraflores]",[Lima],[perú],True,[Doc ID: 7213242c-a1c2-4c33-8d2e-0cb7af3d265e\...
3,https://peru21.pe/lima/atacante-de-miraflores-...,Atacante de Miraflores también agredió a su ex...,Julio Diego Aurelio Solari Zapata fue denuncia...,2025/03/24,"[Miraflores, San Isidro]",[Lima],[perú],True,[Doc ID: 2ee02c81-4598-42ac-9cc7-1cd95773dda6\...
4,https://peru21.pe/mundo/la-senal-de-auxilio-gl...,La señal de auxilio global que puede salvar la...,El sencillo gesto de abrir la palma de la mano...,2025/02/20,[],[],"[canadá, méxico, estados unidos, perú]",True,[Doc ID: 1baa2789-3d06-4f02-bf60-605d14335a26\...


In [29]:
print(textwrap.fill(noticias_raw.document_loader[0][0].text,width=90))

El Congreso aprobó la Ley N° 32351, que modifica la Ley 30364, y obliga a los medios de
comunicación a incluir contenidos educativos contra la violencia familiar y garantizar un
tratamiento respetuoso de las víctimas. Los medios de comunicación del país están
obligadosa participar de la lucha contra laviolencia hacia las mujeresy los integrantes
del grupo familiar. Así lo establece la Ley N° 32351, promulgada por el Congreso y
difundida este martes en El Peruano. La norma modifica el artículo 46 de la Ley 30364, e
introduce disposiciones para que los medios garanticen un enfoque de respeto a los
derechos humanos, la libertad y la dignidad de las víctimas de violencia. También exige
especial cuidado en el tratamiento gráfico de los contenidos que aborden estos casos. Uno
de los principales cambios es la obligación de ceder, de forma gratuita, el 10 % de su
programación durante el horario de protección familiar a instituciones públicas del
Sistema Nacional para la Prevención, Sanción y E

In [30]:
noticias_raw.document_loader[0][0]

Document(id_='b44ade9d-994d-44df-8b32-1bc358fe410a', embedding=None, metadata={'url': 'https://peru21.pe/peru/medios-deberan-tener-franja-educativa-para-apoyar-lucha-contra-la-violencia-hacia-las-mujeres/', 'title': "Medios deberán tener 'franja educativa' para apoyar lucha contra la violencia hacia las mujeres ", 'author': 'Perú 21', 'og:url': 'https://peru21.pe/peru/medios-deberan-tener-franja-educativa-para-apoyar-lucha-contra-la-violencia-hacia-las-mujeres/', 'robots': 'follow', 'og:type': 'article', 'abstract': 'Noticias del Perú y del mundo en peru21.pe - Ultimas noticias de política, deportes, espectáculos, fútbol, economía, mundo, tecnología, cultura, ciencia y más...', 'keywords': 'Violencia contra la mujer,Violencia familiar,Medios de comunicación,Libertad de prensa', 'og:image': 'http://peru21.pe/sites/default/efsfiles/styles/amp_1200x675_16_9/public/2025-05/television.jpg?itok=7pZmAQmf', 'og:title': "Medios deberán tener 'franja educativa' para apoyar lucha contra la violen

In [31]:
docs = sum(noticias_raw['document_loader'].tolist(), [])

In [32]:
embeddings = OpenAIEmbedding(model="text-embedding-ada-002")

In [33]:
pipeline = IngestionPipeline(
    transformations=[
        SemanticSplitterNodeParser(embed_model=embeddings, buffer_size=1, breakpoint_percentile_threshold=95),
        TitleExtractor(),
        OpenAIEmbedding(model="text-embedding-ada-002"),
    ]
)

In [34]:
nodes = pipeline.run(documents = docs)

100%|██████████| 3/3 [00:00<00:00,  3.94it/s]


In [35]:
len(nodes)

11

In [36]:
for i in tqdm(range(len(nodes))):
   try:
    palabras = word_tokenize(nodes[i].metadata['keywords'].lower())
    palabras_filtradas = [p for p in palabras if p.isalnum() and p not in stop_words]
    stems1 = [stemmer.stem(palabra) for palabra in palabras_filtradas]

    palabras = word_tokenize(nodes[i].metadata['document_title'].lower())
    palabras_filtradas = [p for p in palabras if p.isalnum() and p not in stop_words]
    stems2 = [stemmer.stem(palabra) for palabra in palabras_filtradas]

    claves = stems1 + stems2

    nodes[i].metadata['keywords'] = claves
   except:
     print(f'no existe alguna metadata necesaria en el nodo {i}')

100%|██████████| 11/11 [00:00<00:00, 220.59it/s]


In [37]:
nodes[0].metadata

{'url': 'https://peru21.pe/peru/medios-deberan-tener-franja-educativa-para-apoyar-lucha-contra-la-violencia-hacia-las-mujeres/',
 'title': "Medios deberán tener 'franja educativa' para apoyar lucha contra la violencia hacia las mujeres ",
 'author': 'Perú 21',
 'og:url': 'https://peru21.pe/peru/medios-deberan-tener-franja-educativa-para-apoyar-lucha-contra-la-violencia-hacia-las-mujeres/',
 'robots': 'follow',
 'og:type': 'article',
 'abstract': 'Noticias del Perú y del mundo en peru21.pe - Ultimas noticias de política, deportes, espectáculos, fútbol, economía, mundo, tecnología, cultura, ciencia y más...',
 'keywords': ['violenci',
  'muj',
  'violenci',
  'famili',
  'medi',
  'comun',
  'libert',
  'prens',
  'ley',
  '32351',
  'oblig',
  'medi',
  'comun',
  'luch',
  'violenci',
  'famili',
  'peru21',
  'epap',
  'div',
  'collection',
  'of',
  'news',
  'featur',
  'and',
  'opinions',
  'comprehensiv',
  'titl',
  'regulations',
  'for',
  'medi',
  'responsibiliti',
  'in',


In [ ]:
vector_store = le(
    es_url="http://34.60.116.171:9200",
    es_user="elastic",
    es_password='504im-7l4jugthbCMSJV',
    index_name="notices2",
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(nodes, storage_context=storage_context)

### Parte Langchain

In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain.schema.runnable import RunnableLambda, RunnableParallel
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnablePassthrough
from langchain_community.utilities.sql_database import SQLDatabase 
from langchain_community.agent_toolkits import SQLDatabaseToolkit 
from langchain_experimental.tools import PythonREPLTool
from psycopg_pool import ConnectionPool
from langgraph.checkpoint.postgres import PostgresSaver
memory = MemorySaver()

In [9]:
## Ejecutar solo la primera vez para crear automaticamente las tablas
## [checkpoint_blobs, checkpoint_migrations, checkpoint_writes, checkpoints] en el esquema public
#Variables de memoria
connection_kwargs = {
    "autocommit": True,
    "prepare_threshold": 0,
}
# Inicializamos la memoria
with ConnectionPool(
    # Example configuration
    conninfo=uribd,
    max_size=20,
    kwargs=connection_kwargs,
) as pool:
    checkpointer = PostgresSaver(pool)
    checkpointer.setup()

In [7]:
distritos_lima = [
    "Ancón",
    "Ate",
    "Barranco",
    "Breña",
    "Carabayllo",
    "Chaclacayo",
    "Chorrillos",
    "Cieneguilla",
    "Comas",
    "El Agustino",
    "Independencia",
    "Jesús María",
    "La Molina",
    "La Victoria",
    "Lince",
    "Los Olivos",
    "Lurigancho",
    "Lurín",
    "Magdalena del Mar",
    "Miraflores",
    "Pachacámac",
    "Pucusana",
    "Pueblo Libre",
    "Puente Piedra",
    "Punta Hermosa",
    "Punta Negra",
    "Rímac",
    "San Bartolo",
    "San Borja",
    "San Isidro",
    "San Juan de Lurigancho",
    "San Juan de Miraflores",
    "San Luis",
    "San Martín de Porres",
    "San Miguel",
    "Santa Anita",
    "Santa María del Mar",
    "Santa Rosa",
    "Santiago de Surco",
    "Surquillo",
    "Villa El Salvador",
    "Villa María del Triunfo"
]


In [8]:
distritos_lima = distritos_lima + [distrito.upper() for distrito in distritos_lima]

In [9]:
departamentos_peru = [
    "Amazonas",
    "Áncash",
    "Apurímac",
    "Arequipa",
    "Ayacucho",
    "Cajamarca",
    "Callao", 
    "Cusco",
    "Huancavelica",
    "Huánuco",
    "Ica",
    "Junín",
    "La Libertad",
    "Lambayeque",
    "Lima",
    "Loreto",
    "Madre de Dios",
    "Moquegua",
    "Pasco",
    "Piura",
    "Puno",
    "San Martín",
    "Tacna",
    "Tumbes",
    "Ucayali"
]


In [10]:
departamentos_peru = departamentos_peru + [departamento.upper() for departamento in departamentos_peru]

In [11]:
def buscar_ordenados(texto, lista):
    encontrados = []

    for lugar in lista:
        patron = r'\b' + re.escape(lugar) + r'\b'
        match = re.search(patron, texto)
        if match:
            encontrados.append((lugar, match.start()))
    
    # Ordenar por posición en el texto
    encontrados.sort(key=lambda x: x[1])
    
    # Aplicar .title() y eliminar duplicados preservando el orden
    vistos = set()
    resultado = []
    for nombre, _ in encontrados:
        nombre_title = nombre.title()
        if nombre_title not in vistos:
            resultado.append(nombre_title)
            vistos.add(nombre_title)
    
    return resultado

In [12]:
llm = ChatOpenAI(
    temperature=0,
    model="gpt-4")

class State(TypedDict):
    question: str
    inicio: Optional[str]
    fin: Optional[str]
    distrito: List[str]
    departamento: List[str]
    context: List[Document]
    keywords: List[str]
    answer: str

In [13]:
#Creamos la conexion a la base de datos
db_data = SQLDatabase.from_uri(uribd)

# Herramienta BD
toolkit_bd = SQLDatabaseToolkit(db=db_data,llm=llm)
tools_bd = toolkit_bd.get_tools()

In [14]:
def get_schema(_):
  return db_data.get_table_info() # Devuelve la información de las tablas de la base de datos.

def run_query(query):
  return db_data.run(query) # Ejecuta un query en la base de datos.

promptsql = ChatPromptTemplate.from_template("""
Basandonos en el esquema del conjunto de tablas siguiente, escribe una consulta SQL que responda a la pregunta del usuario:
    {schema}

    Pregunta: {question}
    Sql Query:
""")


sql_chain = (
    RunnablePassthrough.assign(schema=get_schema) # Usa RunnablePassthrough para agregar el esquema de la base.
    | promptsql # Le pasa el promptsql al modelo de lenguaje (el LLM) para que genere el SQL.
    | llm.bind(stop=["\nSQLResult:"]) # Se detiene cuando encuentre \nSQLResult: para no generar texto demás.
    | StrOutputParser()
)

promptsqlquery = ChatPromptTemplate.from_template(
    """
    Basandonos en el esquema de tabla inferior, pregunta, SQL Query y Respuesta, escribe una respuesta en lenguaje natural:
    {schema}

    Pregunta: {question}
    Sql Query: {query}
    SQL Respuesta: {response}

    """)

full_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
        schema=get_schema,
        response= lambda vars: run_query(vars["query"]),
    )
    | promptsqlquery
    | llm
)


tool_sql = full_chain.as_tool(
    name="busqueda_feminicidios", description="Consulta en la base de datos informacion sobre feminicidios ocurridos en departamentos y provincias del Perú"
)


C:\Users\Omen\AppData\Local\Temp\ipykernel_27652\2925112533.py:44: LangChainBetaWarning: This API is in beta and may change in the future.
  tool_sql = full_chain.as_tool(


In [15]:
python_executor = PythonREPLTool()

In [16]:
@tool
def graficar_desde_texto(data: str, tipo: str = "barras") -> str:
    """
    Genera y ejecuta un gráfico en Python usando matplotlib/pandas a partir de datos en texto.
    Guarda la imagen localmente y la sube a un bucket de GCS. Devuelve la URL pública.
    """
    import_statement = """import pandas as pd
import matplotlib.pyplot as plt
from io import StringIO
"""

    nombre_imagen = f"{uuid.uuid4().hex}.png"
    ruta_salida = f"./{nombre_imagen}"  # Ruta segura en Windows

    instrucciones_guardado = f'plt.savefig("{ruta_salida}")\nplt.close()'

    prompt = f"""
Eres un generador de código Python. A partir de los siguientes datos en texto:
{data}
Genera un bloque de código Python usando pandas y matplotlib para crear un gráfico de tipo '{tipo}'.

Instrucciones:
- Usa pandas y io.StringIO para leer los datos.
- El DataFrame debe llamarse 'df'.
- Usa matplotlib para graficar.
- No uses plt.show().
- No imprimas nada.
- Al final, guarda el gráfico como '{ruta_salida}' usando plt.savefig y luego plt.close().
- Devuelve SOLO el código Python puro, sin explicaciones, sin comentarios, sin comillas triples.
"""

    try:
        codigo_python = llm.invoke(prompt).content.strip()

        # Si ya contiene plt.savefig, no lo añadimos
        if "plt.savefig" in codigo_python or "plt.close" in codigo_python:
            codigo_completo = import_statement + codigo_python
        else:
            codigo_completo = import_statement + codigo_python + "\n" + instrucciones_guardado

        python_executor.invoke(codigo_completo)

        if not os.path.exists(ruta_salida):
            return f" El archivo gráfico no fue generado: {ruta_salida}. Código generado:\n\n{codigo_python}"

        # Subir a GCS
        bucket_name = "mujerapp-imagenes"
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(nombre_imagen)
        blob.upload_from_filename(ruta_salida)

        # Construir URL pública
        url_publica = f"https://storage.googleapis.com/{bucket_name}/{nombre_imagen}"
        return url_publica

    except Exception as e:
        import traceback
        return f" Error al generar o subir gráfico:\n{traceback.format_exc()}"


In [17]:
# ------------------------
# FUNCIONES AUXILIARES
# ------------------------

def format_docs(docs):
    return "\n\n".join(doc.get_content() for doc in docs)

# ------------------------
prompt_tiempo = ChatPromptTemplate.from_messages([
    ("system",  """Extrae todas las fechas del siguiente texto en formato YYYY/MM/DD.
                Reglas:
                - Si se menciona un día, mes y año como "5 de mayo de 2022" debes agregarle un dia mas a la fecha original, devuelve: 2022/05/05, 2022/05/06.
                - Si se menciona solo un mes y año como "julio de 2022", devuelve: 2022/07/01, 2022/07/31.
                - Si se menciona solo un año como "2022", devuelve dos fechas: 2022/01/01, 2022/12/31.
                - Si se menciona un rango como "entre 2020 y 2022", devuelve: 2020/01/01, 2022/12/31.
                - Si no se menciona ninguna fecha entonces regresa la palabra vacío.

                Ejemplos:
                Entrada: "Durante 2022 aumentaron los casos"
                Salida: 2022/01/01, 2022/12/31

                Entrada: "Entre 2021 y 2022 hubo más casos"
                Salida: 2021/01/01, 2022/12/31

                Entrada: "En mayo de 2022 ocurrieron eventos"
                Salida: 2022/05/01, 2022/05/31

                Entrada: "En marzo y julio del 2022"
                Salida: 2022/03/01, 2022/07/31
     
                Entrada: "El 5 de marzo del 2022"
                Salida: 2022/03/05, 2022/03/06
     
                Entrada: "No se menciona nada de fechas"
                Salida: vacío

                Extrae las fechas del siguiente texto:
                """),
    ("human", "{input}")
])


# ------------------------
# CHAINS de metadatos
# ------------------------

chain_tiempo = RunnableLambda(lambda x: {"input": x["question"]}) | prompt_tiempo | llm

agent_metadata = RunnableParallel({
    "time": chain_tiempo
})

# ---------------------------------
# -     EXTRACCIÓN DE METADATA    -
# ---------------------------------

def extract_metadata(state: State) -> State:
    result = agent_metadata.invoke(state)
    fechas_raw = result["time"].content.strip()

    try:
        if fechas_raw.lower() == "vacío":
            inicio, fin = "", ""
        elif "," in fechas_raw:
            inicio, fin = [f.strip() for f in fechas_raw.split(',')]
        else:
            fecha = datetime.strptime(fechas_raw, "%Y/%m/%d")
            inicio = fecha.strftime("%Y/%m/%d")
            fin = (fecha + timedelta(days=1)).strftime("%Y/%m/%d")
    except Exception:
        inicio, fin = "", ""


    distrito = buscar_ordenados(state['question'], distritos_lima)
    departamento = buscar_ordenados(state['question'], departamentos_peru)

    if inicio and fin:
        state["inicio"] = inicio
        state["fin"] = fin
    if len(distrito)!= 0:
        state["distrito"] = distrito
    if len(departamento)!= 0:
        state['departamento'] = departamento


    palabras = word_tokenize(state['question'].lower())
    palabras_filtradas = [p for p in palabras if p.isalnum() and p not in stop_words]
    keys = [stemmer.stem(palabra) for palabra in palabras_filtradas]
    state['keywords'] = keys

    return state

# ------------------------
# FILTRADO Y RECUPERACIÓN
# ------------------------

def retrieve_docs(state: State) -> State:
    print(state)
    must_filters = []
    if state.get("distrito"):
        must_filters.append({"terms": {"metadata.distrito.keyword": state["distrito"]}})
    if state.get("departamento"):
        must_filters.append({"terms": {"metadata.departamento.keyword": state["departamento"]}})
    if state.get("inicio") and state.get("fin"):
        must_filters.append({
            "range": {
                "metadata.fecha_publicacion": {
                    "gte": datetime.strptime(state["inicio"], "%Y/%m/%d"),
                    "lte": datetime.strptime(state["fin"], "%Y/%m/%d")
                }
            }
        })

    filtro = {
    "bool": {
        "must": must_filters,
        "should": [
            {"terms": {"metadata.keywords.keyword": state["keywords"]}}
        ],
        "minimum_should_match": 0 
        }
    }


    vector_store = le(
    es_url="http://34.60.116.171:9200",
    es_user="elastic",
    es_password='504im-7l4jugthbCMSJV',
    index_name="notices"
    )

    storage_context_read = StorageContext.from_defaults(vector_store=vector_store)

    index_read = VectorStoreIndex.from_vector_store(
        vector_store=vector_store,
        storage_context=storage_context_read,embed_model=OpenAIEmbedding(model="text-embedding-ada-002")
    )
    retriever = index_read.as_retriever(filter=filtro, k=10)
    results = retriever.retrieve(state["question"])
    print(len(results))
    print(results)
    return {**state, "context": results}

# ------------------------
# PROMPT QA
# ------------------------

PROMPT_QA = ChatPromptTemplate.from_template("""
    Eres un asistente para tareas de respuesta a preguntas.
    Usa los siguientes fragmentos de contexto recuperado para responder la pregunta.
    Si no sabes la respuesta, simplemente di que no lo sabes.
    Usa un máximo de tres oraciones y mantén la respuesta concisa.
    
    Pregunta: {question}
    Contexto: {context}
""")

qa_chain = (
    {
        "context": lambda x: format_docs(x["context"]),
        "question": lambda x: x["question"],
    }
    | PROMPT_QA
    | llm
    | StrOutputParser()
)

# ------------------------------------------
# -         HERRAMIENTA PRINCIPAL          -
# ------------------------------------------

@tool
def rag_con_metadata(question: str) -> str:
    """
    Herramienta que responde preguntas sobre violencia de genero.
    """
    print('se usa la herramienta rag')
    initial_state: State = {
        "question": question,
        "inicio": None,
        "fin": None,
        "distrito": [],
        "departamento": [],
        "context": [],
        'keywords': [],
        "answer": ""
    }
    estado_con_metadata = extract_metadata(initial_state) # todo OK ESTA ACA
    estado_con_contexto = retrieve_docs(estado_con_metadata)
    respuesta = qa_chain.invoke(estado_con_contexto)

    estado_con_contexto["answer"] = respuesta
    return respuesta

# ------------------------
# AGENTE FINAL
# ------------------------

tools = [rag_con_metadata,tool_sql,graficar_desde_texto]
modelo = ChatOpenAI(temperature=0)


general_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", f"""Eres un asistente amable y especializado en noticias sobre violencia de género ocurridas en el Perú.
            Tu objetivo es guiar al usuario de forma amigable, breve y conversacional, como si fueras un amigo experto en el tema. Sigue estos lineamientos:
            1. Saludo y contexto: Da un saludo cálido, indícale que eres un agente que cuenta con información de noticias sobre violencia de género ocurridas entre 2019 y 2025. Ademas de que solo puedes consultar en noticias que hayan ocurrido en distritos de Lima y departamentos del Perú. Y pregunta qué desea saber el usuario. 
            2. Estas noticias fueron recopiladas de medios digitales confiables y reconocidos por la ciudadanía como Peru 21, Correo, El Comercio y Exitosa.
            3. Si no sabes la respuesta, dilo con honestidad y ofrece continuar la búsqueda juntos.
            4. Si la pregunta se relaciona con violencia de género, feminicidios, abuso sexual, agresión, hostigamiento, maltrato físico o psicológico, entonces debes usar la herramienta `rag_con_metadata` para responder. 
            5. Si la pregunta se relaciona con consultar a una base de datos, entonces usa la herramienta 'tool_sql'
            6. Si la pregunta se relaciona con generar un grafico, usa primero la herramient 'tool_sql' y luego 'graficar_desde_texto'.
            Es muy importante que siempre uses una herramienta.
            """),

     MessagesPlaceholder(variable_name="messages")

     ]
)

agent2 = create_react_agent(modelo, tools, checkpointer=memory, prompt=general_prompt)


In [21]:
config = {"configurable": {"thread_id": "111"}}
response = agent2.invoke(
    {"messages": [HumanMessage(content="¿Entre el 2020 y 2023, hubo alguna ley que implementara medios digitales que permitan a las mujeres denunciar sus casos de violencia?")]},
    config=config
)

se usa la herramienta rag
{'question': '¿Entre el 2020 y 2023, hubo alguna ley que implementara medios digitales que permitan a las mujeres denunciar sus casos de violencia?', 'inicio': '2020/01/01', 'fin': '2023/12/31', 'distrito': [], 'departamento': [], 'context': [], 'keywords': ['2020', '2023', 'algun', 'ley', 'implement', 'medi', 'digital', 'permit', 'mujer', 'denunci', 'cas', 'violenci'], 'answer': ''}
2
[NodeWithScore(node=TextNode(id_='45a2bc59-0188-4e3c-996b-fa23f44c2b99', embedding=None, metadata={'url': 'https://peru21.pe/peru/resuelven-mas-de-170-mil-denuncias-de-violencia-contra-la-mujer-en-juzgados-habilitados-con-expediente-electronico-peru-expediente-electronico-eje-poder-judicial-noticia/', 'title': 'Resuelven más de 170 mil denuncias de violencia contra la mujer en juzgados habilitados con expediente electrónico', 'author': 'Perú 21', 'og:url': 'https://peru21.pe/peru/resuelven-mas-de-170-mil-denuncias-de-violencia-contra-la-mujer-en-juzgados-habilitados-con-expedien

In [22]:
print(textwrap.fill(response["messages"][-1].content,width=90))

Entre el 2020 y 2023 se implementó la Ley 31156 en el Perú, la cual permitió a las mujeres
denunciar casos de violencia a través de medios digitales. Esta ley modificó el artículo
15 de la Ley 30364, habilitando el uso permanente de canales tecnológicos para denunciar
hechos de violencia. Además, se implementaron juzgados de familia con el Expediente
Judicial Electrónico (EJE) y la Mesa de Partes Electrónica (MPE) desde el año 2022 para
gestionar estos casos. ¿Hay algo más en lo que pueda ayudarte?


In [64]:
config = {"configurable": {"thread_id": "111"}}
response = agent2.invoke(
    {"messages": [HumanMessage(content="Consulta en la base de datos el numero de casos en los cuales el parentesco con la victima no de sea esposos y genera un grafico de pie donde cada seccion sea un departamento, omite las departamentos que no tengan ningun caso")]},
    config=config
)

In [65]:
print(textwrap.fill(response["messages"][-1].content,width=90))

Según la base de datos, hay un total de 134 casos en los que el parentesco con la víctima
no es el de esposo. A continuación, te muestro un gráfico de pie donde cada sección
representa un departamento y se omiten los departamentos que no tienen ningún caso:
![Gráfico de Pie](https://storage.googleapis.com/mujerapp-
imagenes/30dc5b149d5c40518ff00efeec02883a.png)  Si necesitas más información o tienes
alguna otra consulta, ¡estaré aquí para ayudarte!


In [ ]:
config = {"configurable": {"thread_id": "111"}}
response = agent2.invoke(
    {"messages": [HumanMessage(content="¿Cuantos casos de abuso sexual se registraron en Cusco entre enero y febrero de 2023?")]},
    config=config
)

In [ ]:
print(textwrap.fill(response["messages"][-1].content,width=90))

### RAGAS

In [22]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from ragas.llms import LangchainLLMWrapper
evaluator_llm = LangchainLLMWrapper(modelo)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001BE11853DA0>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x000001BE11D09730>, 81297.546)]']
connector: <aiohttp.connector.TCPConnector object at 0x000001BE11853860>


In [23]:
sample_queries = [
    "¿Entre el 2020 y 2023, hubo alguna ley que implementara medios digitales que permitan a las mujeres denunciar sus casos de violencia?",
    "Que acciones el Ministerio Público impulsa en Piura, cuales son las cifras de violencia dentro de piura y dime su comparacion con otros departamentos. Detalla cada punto",
    "¿Cuantos casos de abuso sexual se registraron en Cusco entre enero y febrero de 2023?"
]

expected_responses = [
    "Sí, entre 2020 y 2023 se implementó la Ley N.º 31156 (publicada el 8 de abril de 2021), que modificó la Ley 30364 para habilitar de forma permanente el uso de canales tecnológicos como correos electrónicos, mensajería instantánea y otros medios digitales para denunciar casos de violencia contra las mujeres y los integrantes del grupo familiar, sin requerir pruebas físicas ni representación legal.",
    "El Ministerio Publico busca la necesidad de coordinar acciones conjuntas y estrategias efectivas para optimizar los tiempos de respuesta ante denuncias y brindar un acompañamiento más oportuno a las víctimas. Asimismo, la región Piura registra 6275 de violencia, de los cuales 5580 son mujeres. El mayor número de denuncias son de mujeres entre los 18 y 35 años (1537) y la violencia psicológica con 3000 casos, le sigue la física con 2342, sexual con 926 y económica con 7 denuncias. Del total de denuncias registradas, 1769 fueron agresiones físicas severas, 3037 medianas y 1469 leves. De las 305 denuncias por violencia sexual, 288 corresponden a mujeres y la mayoría de agraviadas está entre los 0 y 17 años (198). Y según los últimos cinco años, la región Piura registra 33948 casos de violencia, ocupando el sexto lugar a nivel nacional, seguido de Lima (201481), Arequipa (72075), Cusco (49553), Ancash (37182) y La Libertad ( 35515).",
    "Se ha registrado 34 casos de abuso sexual en Cusco."
]

In [24]:
# Dataset de RAGAS
ragas_samples = []

for query, reference in zip(sample_queries, expected_responses):
    # Ejecutar tu RAG con metadatos
    initial_state: State = {
        "question": query,
        "inicio": None,
        "fin": None,
        "distrito": [],
        "departamento": [],
        "context": [],
        "keywords": [],
        "answer": ""
    }

    estado_con_metadata = extract_metadata(initial_state)
    estado_con_contexto = retrieve_docs(estado_con_metadata)
    respuesta = qa_chain.invoke(estado_con_contexto)

    # Agregar al dataset RAGAS
    ragas_samples.append({
        "question": query,
        "answer": respuesta,
        "contexts": [doc.get_content() for doc in estado_con_contexto["context"]],
        "reference": [reference]
    })

# Convertir a HuggingFace Dataset
hf_dataset = Dataset.from_list(ragas_samples)

{'question': '¿Entre el 2020 y 2023, hubo alguna ley que implementara medios digitales que permitan a las mujeres denunciar sus casos de violencia?', 'inicio': '2020/01/01', 'fin': '2023/12/31', 'distrito': [], 'departamento': [], 'context': [], 'keywords': ['2020', '2023', 'algun', 'ley', 'implement', 'medi', 'digital', 'permit', 'mujer', 'denunci', 'cas', 'violenci'], 'answer': ''}
2
[NodeWithScore(node=TextNode(id_='45a2bc59-0188-4e3c-996b-fa23f44c2b99', embedding=None, metadata={'url': 'https://peru21.pe/peru/resuelven-mas-de-170-mil-denuncias-de-violencia-contra-la-mujer-en-juzgados-habilitados-con-expediente-electronico-peru-expediente-electronico-eje-poder-judicial-noticia/', 'title': 'Resuelven más de 170 mil denuncias de violencia contra la mujer en juzgados habilitados con expediente electrónico', 'author': 'Perú 21', 'og:url': 'https://peru21.pe/peru/resuelven-mas-de-170-mil-denuncias-de-violencia-contra-la-mujer-en-juzgados-habilitados-con-expediente-electronico-peru-expedi

In [ ]:
hf_dataset

NameError: name 'hf_dataset' is not defined

: 

In [25]:
hf_dataset = hf_dataset.map(lambda x: { "reference": x["reference"][0] if isinstance(x["reference"], list) else x["reference"] })

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [26]:
results = evaluate(
    hf_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],llm=evaluator_llm)

print(results)


Evaluating:   0%|          | 0/9 [00:00<?, ?it/s]

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001BE14D9B8C0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001BE7EA9DEE0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001BE14E36FF0>


{'context_recall': 0.7222, 'faithfulness': 0.8571, 'factual_correctness(mode=f1)': 0.4400}
